In [106]:
#----Tableau REST API-----
import tableauserverclient as TSC
import pandas as pd

# Credentials and Sitename
token_name = ''
token_value = ''
sitename = ''
server_url = ''

tableau_auth = TSC.PersonalAccessTokenAuth(token_name,token_value,sitename)
server = TSC.Server(server_url,use_server_version=True)

### Datasources

In [67]:
with server.auth.sign_in(tableau_auth):
    all_datasources, pagination_item = server.datasources.get()
    print("\nThere are {} datasources on site: ".format(pagination_item.total_available))
    data_source_list = []
    for datasource in all_datasources:
        data_source_list.append((datasource.id,datasource.name))
df_datasources = pd.DataFrame(data_source_list,columns=['datasource_id','datasource_name'])
df_datasources.head(3)


There are 49 datasources on site: 


,datasource_id,datasource_name
0,63878ad5-1118-4ffa-95f2-eec253555930,B2BBookings_TableauLink_SBT_GDS (B2BSalesRepor...
1,f24fe520-50e5-42e0-8b0f-f7d26e49bc86,B2BBookings_TableauLink_SBT_GDS_Ancillaries (B...
2,d44a3e09-7a94-41f9-953b-fab01f215f4b,v_B2BBookings_Scrapers (B2BSalesReporting)


### Projects

In [68]:
with server.auth.sign_in(tableau_auth):
    all_projects, pagination_item = server.projects.get()
    print("\nThere are {} projects on site: ".format(pagination_item.total_available))
    project_list = []
    for project in all_projects:
        project_list.append((project.id,project.name))
df_projects = pd.DataFrame(project_list,columns=['project_id','project_name'])
df_projects.head(3)


There are 78 projects on site: 


,project_id,project_name
0,53a7cdbd-e646-456e-b752-f0820c8f9074,Commercial
1,0a9bc093-6694-47aa-bcb4-ff42cd44bef9,Commercial Restricted
2,a42473f4-2179-4cf6-98da-1f2ebec960d3,Platform Information


### Workbooks

In [69]:
with server.auth.sign_in(tableau_auth):
    workbook_list = []
    for wb in TSC.Pager(server.workbooks):
        workbook_list.append((wb.id,wb.project_id,wb.name))
    print(f"\nThere are {len(workbook_list)} workbooks in site: ")
df_workbooks = pd.DataFrame(workbook_list,columns=['workbook_id','project_id','workbook_name'])
df_workbooks.head(3)


There are 389 workbooks in site: 


,workbook_id,project_id,workbook_name
0,4da60fb7-ccaf-4c77-9c21-b152ed802b4b,2d7586fc-a815-4df7-9dd8-ba6ffaedd229,Scraper API Bookings
1,f609eda3-3231-4717-aef6-6f5ff65f09f3,2d7586fc-a815-4df7-9dd8-ba6ffaedd229,OAG - easyJet Vs EU Network Capacity
2,f6ec309c-a84a-48d5-bbc2-16d1a5f44dad,0c6070ad-d3e5-4a01-ae5d-33436f166df8,Loyalty Dashboard


### Views

In [71]:
with server.auth.sign_in(tableau_auth):
    view_list = []
    for view in TSC.Pager(server.views):
        view_list.append((view.workbook_id,view.id,view.name))
df_views =  pd.DataFrame(view_list,columns=['workbook_id','view_id','view_name'])
df_views.head(3)

,workbook_id,view_id,view_name
0,4da60fb7-ccaf-4c77-9c21-b152ed802b4b,3da2fd6e-bc86-4a64-ba30-c0df2cb5f862,Weekly Summary Chart
1,4da60fb7-ccaf-4c77-9c21-b152ed802b4b,56749fb7-e7f9-4372-972e-4437cbf8702c,Daily Summary Chart Comparison
2,4da60fb7-ccaf-4c77-9c21-b152ed802b4b,752399e1-1ca6-41aa-a221-fb4ab26101b0,Daily Summary Chart


### Merging

In [76]:
df_reportingHub = pd.merge(df_workbooks,df_projects,how='left',on='project_id')
df_reportingHub = pd.merge(df_reportingHub,df_views,how='left',on='workbook_id')
df_reportingHub = df_reportingHub[['project_id','workbook_id','view_id','project_name','workbook_name','view_name']]
df_reportingHub.head(3)

,project_id,workbook_id,view_id,project_name,workbook_name,view_name
0,2d7586fc-a815-4df7-9dd8-ba6ffaedd229,4da60fb7-ccaf-4c77-9c21-b152ed802b4b,3da2fd6e-bc86-4a64-ba30-c0df2cb5f862,Group Markets Restricted,Scraper API Bookings,Weekly Summary Chart
1,2d7586fc-a815-4df7-9dd8-ba6ffaedd229,4da60fb7-ccaf-4c77-9c21-b152ed802b4b,56749fb7-e7f9-4372-972e-4437cbf8702c,Group Markets Restricted,Scraper API Bookings,Daily Summary Chart Comparison
2,2d7586fc-a815-4df7-9dd8-ba6ffaedd229,4da60fb7-ccaf-4c77-9c21-b152ed802b4b,752399e1-1ca6-41aa-a221-fb4ab26101b0,Group Markets Restricted,Scraper API Bookings,Daily Summary Chart


### Project/Folder Selection

In [136]:
folder = 'Safety, Security & Compliance'
folder_select = df_reportingHub.loc[df_reportingHub['project_name'] == folder]
folder_select_id = folder_select.iat[0,0]
print(f'The project_id of {folder} is: {folder_select_id}')
df_tableau_safety = df_reportingHub.loc[df_reportingHub['project_id'] == folder_select_id]
df_tableau_safety.head(5)

The project_id of Safety, Security & Compliance is: ee77b47e-ad0b-4d0a-b2ed-1dd35bfebf04


,project_id,workbook_id,view_id,project_name,workbook_name,view_name
642,ee77b47e-ad0b-4d0a-b2ed-1dd35bfebf04,478e0d72-ef82-44bd-9fb1-5b18a4e57885,3162fea9-a234-4971-a490-facd96273156,"Safety, Security & Compliance",Ground Ops SPI Tool,GROUND OPS SPI TOOL
643,ee77b47e-ad0b-4d0a-b2ed-1dd35bfebf04,478e0d72-ef82-44bd-9fb1-5b18a4e57885,222954fc-8e62-4202-8e4b-7e9ed4cb652f,"Safety, Security & Compliance",Ground Ops SPI Tool,SEASONAL ANALYSIS DASHBOARD
693,ee77b47e-ad0b-4d0a-b2ed-1dd35bfebf04,02ae0d81-9019-4dc5-8407-a7afaf59b1d9,3dcac6bc-98d0-46c7-9cf3-0b1514c97f8c,"Safety, Security & Compliance",GOST V2,Ground Operations SPI Tool - Risk Dashboard
694,ee77b47e-ad0b-4d0a-b2ed-1dd35bfebf04,02ae0d81-9019-4dc5-8407-a7afaf59b1d9,ba83065d-9469-46fb-a7c5-105bfbf5f8a6,"Safety, Security & Compliance",GOST V2,Dashboard 2
848,ee77b47e-ad0b-4d0a-b2ed-1dd35bfebf04,e280cfb6-b1bf-4a19-8ea4-02728d4ac0eb,258794a0-bbe9-4926-99a4-e5ca949e41cc,"Safety, Security & Compliance",Audits and Findings,External Audits and Findings


### Saving Selected View

In [135]:
# Selecting relevant index of dashboard view
view_index = 1301
view_id = df_tableau_safety.at[view_index,'view_id']
view_name = df_tableau_safety.at[view_index,'view_name']

# Image options
view_image_options = TSC.ImageRequestOptions(imageresolution=
                    TSC.ImageRequestOptions.Resolution.High, maxage=1)

with server.auth.sign_in(tableau_auth):

    # Retrieving requested view by id number
    request_view = server.views.get_by_id(view_id)
    print('The requested view is: ',request_view.id,request_view.name)

    #Saving the requested view
    server.views.populate_image(request_view)
    with open('Company Reporting.png','wb') as v:
        v.write(request_view.image)
    print('Tableau View has been Saved.')

The requested view is:  1efaef2a-51e9-4de5-96ac-695ad0c16806 Company Reporting
Tableau View has been Saved.


### Cropping Desired Sections of Selected View

In [151]:
from PIL import Image

# Grabbing original Image
im = Image.open("Company Reporting.png")

# Setting points for cropped image
left = 0
top = 0
right = 750
bottom = 530

# Cropping to set dimension
im_cropped = im.crop((left,top,right,bottom))
im_cropped.save('Company Reporting_cropped.png')
im_cropped.show()
print('Dashboard Image Has been Cropped')

Dashboard Image Has been Cropped
